### Renegade testing suite
**Testing for perft, search and pruning efficiency**

There are three tests:
- **perft**: whether the move generator is working properly
- **perft speed**: speed of move generation with perft(5)
- **search speed**: time to reach depth 8

Testing positions from  
https://gist.github.com/peterellisjones/8c46c28141c162d1d8a0f0badbc9cff9

In [ ]:
# Set the path to executable and json here
path = "../x64/Release/Renegade.exe"
fens = "perft.json"

# Prints verbose output
verbose = False

In [ ]:
import subprocess
import json
import time

with open(fens) as file:
    data = json.load(file)

def run_perft():
    print("Perft passing:")
    good = 0
    total_nodes = 0;
    total_time_spent = 0;
    i = 0
    for d in data:
        i += 1
        depth = int(d['depth'])
        result = checkperft(d['fen'], depth)
        correct = int(d['nodes'])
        total_nodes += result[0]
        if (result[0] == correct):
            c = "✓"
            good += 1
        else: c = "✗"
        total_time_spent += result[1]
        if verbose:
            print("%2d. %s | %7d | %7d | %d: %s" % (i, c, correct, result[0], depth, d['fen']))
    if good == len(data):
        print("\nPerft passed.")
    else:
        print("\nPerft failed!")
    print("Correct: %d/%d\n\n" % (good, i))
    
def run_perft_speed():
    print("Perft speed:")
    total_nodes = 0;
    total_time_spent = 0;
    i = 0
    for d in data:
        i += 1
        result = checkperft(d['fen'], 5)
        correct = int(d['nodes'])
        total_nodes += result[0]
        total_time_spent += result[1]
        if verbose: 
            print("%d/%d  " % (i, len(data)), end='\r')
    print("\nTime spent: %.3f s" % (total_time_spent/1e9))
    print("Nodes: %d" % (total_nodes))
    print("Engine nps: %.3fm\n\n" % (total_nodes/total_time_spent*1e3))
    
def run_search_speed():
    print("Search speed:")
    i = 0
    total_time = 0
    total_nodes = 0
    for d in data:
        if verbose: 
            print("[Position %d] : %s" % (i+1, data[i]['fen']))
        output_str = checkeval(data[i]['fen'])
        output = output_str.split()
        total_time += int(output[13])
        total_nodes += int(output[9])
        if verbose: 
            print(output_str)
        i += 1
    print("\nTotal nodes: %d" % total_nodes)
    print("Total time: %.2f s" % (total_time/1000))
    print("Engine nps: %.1fk" % (total_nodes / total_time))

def checkperft(fen, d):
    proc = subprocess.Popen([path, '-l', ''], stdout=subprocess.PIPE, stdin=subprocess.PIPE)
    proc.stdin.write(('position fen ' + fen + '\n').encode())
    proc.stdin.write(('go perfd ' + str(d) +'\n').encode())
    proc.stdin.close()
    i = 0
    start = time.time_ns()
    while True:
        i += 1
        k = proc.stdout.readline()
        if k == b'': break
        if i == 2:
            spent = time_spent = time.time_ns()-start
            return int(k.decode()), spent

def checkeval(fen):
    proc = subprocess.Popen([path, '-l', ''], stdout=subprocess.PIPE, stdin=subprocess.PIPE)
    proc.stdin.write(('position fen ' + fen + '\n').encode())
    proc.stdin.write(('go depth 8\n').encode())
    proc.stdin.close()
    i = 0
    start = time.time_ns()
    while True:
        i += 1
        k = proc.stdout.readline()
        if k == b'': break
        if i == 9:
            spent = time_spent = time.time_ns()-start
            return k

In [ ]:
run_perft()
run_perft_speed()
run_search_speed()